In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import balanced_accuracy_score

from pprint import pprint

In [11]:
df = pd.read_csv('../data/pred_herg/pred_herg_dataset_maccsfp.csv')

labels = df['activity']
features = df.drop('activity', axis = 1)

features = np.array(features)
labels = np.array(labels)

In [12]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, 
                                                                            test_size = 0.3, random_state = 42)

print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (4188, 153)
Training Labels Shape: (4188,)
Testing Features Shape: (1796, 153)
Testing Labels Shape: (1796,)


In [4]:
rf = RandomForestRegressor(random_state = 42)

print('Parameters currently in use:\n')
pprint(rf.get_params())

Parameters currently in use:

{'bootstrap': True,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 'warn',
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}


### Random Search with CV

In [16]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 100, num = 10)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]}


In [17]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier(random_state = 42)
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator=rf, param_distributions=random_grid,
                              n_iter = 100, scoring='neg_mean_absolute_error', 
                              cv = 5, verbose=2, random_state=42, n_jobs=-1,
                              return_train_score=True)

# Fit the random search model
rf_random.fit(train_features, train_labels);

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   16.2s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  5.3min finished


In [18]:
rf_random.best_params_

{'n_estimators': 1000,
 'min_samples_split': 2,
 'min_samples_leaf': 2,
 'max_features': 'sqrt',
 'max_depth': 100,
 'bootstrap': False}

In [20]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    labels = np.round(predictions,0)
    bacc = balanced_accuracy_score(test_labels, labels)
    bacc = float("{0:.2f}".format(bacc))
    
    # claculate sensitivity and specificity
    confusion = confusion_matrix(test_labels, labels)
    TP = confusion[1, 1]
    TN = confusion[0, 0]
    FP = confusion[0, 1]
    FN = confusion[1, 0]
    
    sensitivity = TP / float(FN + TP)
    sensitivity = float("{0:.2f}".format(sensitivity))
    
    specificity = TN / float(TN + FP)
    specificity = float("{0:.2f}".format(specificity))
    
    auc = roc_auc_score(test_labels, predictions)
    auc = float("{0:.2f}".format(auc))
    
    return auc, bacc, sensitivity, specificity

In [37]:
# base model using default params

base_model = RandomForestClassifier(n_estimators = 100, random_state = 42)
base_model.fit(train_features, train_labels)
performance = evaluate(base_model, test_features, test_labels)
print(performance)

(0.77, 0.77, 0.87, 0.67)


In [22]:
# best model using random search cv

best_random = rf_random.best_estimator_
performance = evaluate(best_random, test_features, test_labels)
print(performance)

(0.77, 0.77, 0.88, 0.66)


### Grid Search with CV

In [29]:
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [False],
    'max_depth': [80, 90, 100, 110],
    'max_features': ['sqrt'],
    'min_samples_leaf': [2, 3, 4],
    'min_samples_split': [2, 4, 6],
    'n_estimators': [100, 500, 1000]
}

# Create a base model
rf = RandomForestClassifier(random_state = 42)

# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 5, n_jobs = -1, verbose = 2, return_train_score=True)

grid_search.fit(train_features, train_labels);

Fitting 5 folds for each of 108 candidates, totalling 540 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   14.4s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 540 out of 540 | elapsed:  5.5min finished


In [30]:
grid_search.best_params_

{'bootstrap': False,
 'max_depth': 80,
 'max_features': 'sqrt',
 'min_samples_leaf': 2,
 'min_samples_split': 2,
 'n_estimators': 1000}

In [39]:
best_grid = grid_search.best_estimator_
performance = evaluate(best_grid, test_features, test_labels)
print(performance)

(0.77, 0.77, 0.88, 0.66)


In [40]:
final_model = grid_search.best_estimator_

print('Final Model Parameters:\n')
pprint(final_model.get_params())

Final Model Parameters:

{'bootstrap': False,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 80,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 2,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 1000,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}


In [45]:
# read external test set

external = pd.read_csv('../data/ncats/hERG_data_maccsfp.csv')

ext_labels = external['activity']
ext_features = external.drop('activity', axis = 1)
#ext_features = ext_features.fillna(train_features.mean())

ext_features = np.array(ext_features)
ext_labels = np.array(ext_labels)

print('External Features Shape:', ext_features.shape)
print('External Labels Shape:', ext_labels.shape)

External Features Shape: (1838, 153)
External Labels Shape: (1838,)


In [47]:
# apply the best model built with best params on the external test set

base_model = RandomForestClassifier(n_estimators = 100, random_state = 42)
base_model.fit(features, labels)

performance = evaluate(base_model, ext_features, ext_labels)
print(performance)

(0.51, 0.51, 0.82, 0.2)
